In [1]:
## TAREA 1 ENZO ANTILEF, ARTURO MORALES.

from finrisk import QC_Financial_3 as Qcf #PRIMERO IMPORTAMOS LOS PAQUETES A USAR PARA LA TAREA. 
from dataclasses import dataclass
from enum import Enum
from IPython.display import Image
from IPython.core.display import HTML
import pandas as pd
from datetime import date
import numpy as np
import sys
from functools import partial
from scipy.optimize import root_scalar

# Modificado por AD
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [2]:
class BusCal(Enum): #DEFINIMOS EL CALENDARIO DE CADA PAÍS.
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:
    """ 
    """
    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

In [4]:
# IMPORTAMOS LA BASE DE DATOS A UTILIZAR.
df = pd.read_excel('../data/SOFR-10012019-10292020.xls',"nice format")

df['fecha_inicial'] = df['DATE'] #DEFINIMOS UNA COLUMNA DE LA BASE CON LAS FECHAS INICIALES.

df = df.drop(df.columns[[1]], axis='columns') #BOTAMOS LA COLUMNA QUE REPLICAMOS.
fecha = df.iloc[0:269,0] 
addfecha = "2020-10-29"
fecha = np.append(addfecha, fecha)
df["fecha_final"]= pd.Series(fecha)
df['tasa'] = df['RATE\n(PERCENT)'] * 0.01
df= df.iloc[::-1] #INVERTIMOS LAS FILAS PARA QUE ESTÉN ORDENADAS DESDE EL DATO MÁS ANTIGUO AL MÁS RECIENTE.
df['fecha_inicial'] = pd.to_datetime(df['fecha_inicial'], errors='coerce')
df['fecha_final'] = pd.to_datetime(df['fecha_final'], errors='coerce')
df['fecha_inicial'] = df['fecha_inicial'].dt.date
df['fecha_final'] = df['fecha_final'].dt.date
df = df.drop(df.columns[[1]], axis='columns')
df = df.drop(df.columns[[0]], axis='columns')



print(df)

    fecha_inicial fecha_final    tasa
269    2019-10-01  2019-10-02  0.0188
268    2019-10-02  2019-10-03  0.0185
267    2019-10-03  2019-10-04  0.0184
266    2019-10-04  2019-10-07  0.0182
265    2019-10-07  2019-10-08  0.0183
..            ...         ...     ...
4      2020-10-22  2020-10-23  0.0007
3      2020-10-23  2020-10-26  0.0008
2      2020-10-26  2020-10-27  0.0009
1      2020-10-27  2020-10-28  0.0009
0      2020-10-28  2020-10-29  0.0008

[270 rows x 3 columns]


In [5]:
class OisCashFlow:
    
        #UTILIZAMOS _INIT_ PARA GUARDAR LOS INPUTS Y OCUPARLOS CON LA NOTACIÓN "SELF" EN LAS FUNCIONES DE LA CLASE
        
        # AD: esto no era necesario. El decorador @dataclass ya proveía esta estructura.
    
    def __init__ (self, start_date, end_date, settlement_date, notional, currency, amortization,amort_is_cashflow,interest_rate, on_index, spread, gearing):
        
        self.start_date = start_date
        self.end_date = end_date
        self.settlement_date = settlement_date
        self.notional = notional
        self.currency = currency
        self.amortization = amortization
        self.amort_is_cashflow = amort_is_cashflow
        self.interest_rate = interest_rate
        self.on_index = on_index
        self.spread = spread
        self.gearing = gearing
        
        
        
        #DEFINIMOS LA FUNCIÓN QUE VA A CALCULAR LA TASA EQUIVALENTE

    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixing: Qcf.time_series) -> float:
        
        vector_fech = []
        Fechas_finales = Qcf.time_series()               
        self.fecha_final = accrual_date
        Dif_dias = self.start_date.day_diff(self.fecha_final) #DIFERENCIA ENTRE FECHA INICIAL Y FECHA FINAL.
        
        
        #HACEMOS UN BUCLE PARA PODER RESCATAR LAS FECHAS Y LAS TASAS ASOCIADAS A ESTAS MEDIANTE EL FIXING.

        for i in range(270):


                        x = Qcf.QCDate(df.iloc[i]['fecha_inicial'].day,
                                            df.iloc[i]['fecha_inicial'].month,
                                            df.iloc[i]['fecha_inicial'].year)
                        y = Qcf.QCDate(df.iloc[i]['fecha_final'].day,
                                            df.iloc[i]['fecha_final'].month,
                                            df.iloc[i]['fecha_final'].year)
                        vector_fech = i

                        if x == self.start_date:

                             for j in range(vector_fech,vector_fech+Dif_dias):

                                z = Qcf.QCDate(df.iloc[j]['fecha_inicial'].day,
                                            df.iloc[j]['fecha_inicial'].month,
                                            df.iloc[j]['fecha_inicial'].year)

                                w = Qcf.QCDate(df.iloc[j]['fecha_final'].day,
                                            df.iloc[j]['fecha_final'].month,
                                            df.iloc[j]['fecha_final'].year)

                                fixing[z] = df.iloc[j]['tasa']
                                Fechas_finales[w] = df.iloc[j]['tasa']

        F_fin_fix = Qcf.time_series_dates(Fechas_finales)  #FECHAS FINALES CORRESPONDIENTES AL FIXING.
        Tasas_fix = Qcf.time_series_values(fixing)         #TASAS CORRESPONDIENTES A LAS FECHAS MEDIANTE EL FIXING.
        F_in_fix = Qcf.time_series_dates(fixing)           #FECHAS CORRESPONDIENTES AL FIXING.                                
        


        #TABULAMOS LOS 3 ELEMENTOS DEL FIXING EN EL SIGUIENTE DATA FRAME.
        datos_fixing= pd.DataFrame({'Fechas iniciales fixing':F_in_fix,'Fechas finales fixing':F_fin_fix,'Tasas diarias fixing':Tasas_fix})

        
                
        #LLAMAMOS A LA FUNCIÓN QUE PERMITE OBTENER LOS FACTORES DE CAPITALIZACIÓN 
        def get_wf(fecha_inicial: self.start_date, fecha_final: self.fecha_final, valor_tasa: float) -> float:
                            self.qc_tasa = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())    
                            return self.qc_tasa.wf(fecha_inicial, fecha_final)

        datos_fixing['wf'] = datos_fixing.apply(
        lambda row: get_wf(row['Fechas iniciales fixing'], row['Fechas finales fixing'],row['Tasas diarias fixing']),
        axis=1
        )

        datos_fixing.style.format({'Tasas diarias fixing': '{0:.2%}', 'wf': '{0:.6%}'})
       
     
        
        
        self.P = datos_fixing['wf'].product() #PRODUCTO DE LOS FACTORES DE CAPITALIZACIÓN.
        self.Dif_dias = self.start_date.day_diff(self.fecha_final)
        self.tasa_eq = (self.P - 1) * 360.0 / self.Dif_dias #CÁLCULO TASA EQUIVALENTE.

        return self.tasa_eq

 

        #FUNCIÓN QUE PERMITE CALCULAR EL MONTO TOTAL DE LOS INTERESES.
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        
        self.interes_ois = self.notional * self.tasa_eq * self.Dif_dias / 360     
        return self.interes_ois       
    
    
    
        
        #FUNCIÓN QUE PERMITE CALCULAR EL NOCIONAL+INTERESES.
    def amount(self, fixings: Qcf.time_series) -> float:
        self.qc_tasa_eq = Qcf.QCInterestRate(self.tasa_eq, Qcf.QCAct360(), Qcf.QCLinearWf())
        self.flujo = self.amortization + self.notional * (self.qc_tasa_eq.wf(self.start_date, self.fecha_final) - 1)
        return self.flujo


In [6]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)



#ENTREGAMOS VALORES ARBITRARIOS A LA CLASE DEFINIDA PARA OBTENER LOS VALORES DE LAS FUNCIONES SEGÚN ESTOS.
ois = OisCashFlow(
    Qcf.QCDate(4,10,2019),
    Qcf.QCDate(4, 1, 2020),
    Qcf.QCDate(4, 1, 2020),
    100000000,
    Qcf.QCUSD(),
    100000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)
fixing =  Qcf.time_series()

In [7]:
print(f"Tasa equivalente es igual a: {ois.get_accrued_rate(Qcf.QCDate(2,1,2020),fixing):.4%}")
print(f"Intereses devengados son iguales a: {ois.get_accrued_interest(Qcf.QCDate(2,1,2020),fixing)}")
print(f"Monto total dado el nocional escogido :{ois.amount(fixing)}")

Tasa equivalente es igual a: 2.4757%
Intereses devengados son iguales a: 618933.8396277444
Monto total dado el nocional escogido :100618933.83962774


In [8]:
#IMPORTAMOS BASE DE DATOS PARA PODER CALCULAR LA CURVA CERO CUPÓN.
df4 = pd.read_excel('../data/20201012_built_sofr_zero.xlsx') 


#CÁLCULO CURVA CERO CUPÓN.
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df4)
frmt = {
    'tasa': '{:.6%}',
    'df': '{:.6%}',
    'valor_tasa': '{:.4%}',
    'spread': '{:.4%}',
    'nominal': '{:,.2f}',
    'interes': '{:,.2f}',
    'amortizacion': '{:,.2f}',
    'flujo': '{:,.2f}',
}

In [9]:
#FUNCIÓN QUE ENTREGA LOS VALORES ESPERADOS DE TASAS NOMINALES ANUALES DEPENDIENDO DEL PLAZO A ESCOGER.
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois: ois,
        zcc: Qcf.ZeroCouponCurve,
        fixings: Qcf.time_series,
        fecha_final: Qcf.QCDate) -> None:
    
        
        vector_fech = []
        Fechas_finales = Qcf.time_series()               
        fecha_final = fecha_final
        Dif_dias = val_date.day_diff(fecha_final)

        for i in range(270):


                        x = Qcf.QCDate(df.iloc[i]['fecha_inicial'].day,
                                            df.iloc[i]['fecha_inicial'].month,
                                            df.iloc[i]['fecha_inicial'].year)
                        y = Qcf.QCDate(df.iloc[i]['fecha_final'].day,
                                            df.iloc[i]['fecha_final'].month,
                                            df.iloc[i]['fecha_final'].year)
                        vector_fech = i

                        if x == val_date:

                             for j in range(vector_fech,vector_fech+Dif_dias):

                                z = Qcf.QCDate(df.iloc[j]['fecha_inicial'].day,
                                            df.iloc[j]['fecha_inicial'].month,
                                            df.iloc[j]['fecha_inicial'].year)

                                w = Qcf.QCDate(df.iloc[j]['fecha_final'].day,
                                            df.iloc[j]['fecha_final'].month,
                                            df.iloc[j]['fecha_final'].year)




                                fixing[z] = df.iloc[j]['tasa']
                                Fechas_finales[w] = df.iloc[j]['tasa']

        F_fin_fix = Qcf.time_series_dates(Fechas_finales)                     
        F_in_fix = Qcf.time_series_dates(fixing)                                     
        Tasas_fix = Qcf.time_series_values(fixing)
                  
        
        get_ois_sofr = partial(aux.get_ois_using_template, aux.type_ois_template, aux.TypeOis.SOFR)
     
        op = get_ois_sofr(
        rp=Qcf.RecPay.RECEIVE,  #RECIBIMOS PATA FIJA.
        notional=ois.notional,
        start_date=ois.start_date,
        tenor=Qcf.Tenor('6Y'),  #PLAZO CONTRATO A 6 AÑOS.
        fixed_rate_value=.01,   #TASA FIJA DE 0.01
        spread=ois.spread,
        gearing=ois.gearing)
    
        fwd = Qcf.ForwardRates()
        for n in Tasas_fix:
            tabla = fwd.set_rates_icp_clp_leg(val_date, n, op[1], zcc)
            return (aux.show_leg(op[1], 'IcpClpCashflow', '').style.format(frmt), tabla)


        

In [10]:
## VEMOS QUE LOS MONTOS SON NEGATIVOS PORQUE ELEGIMOS PAGAR LA PATA FLOTANTE, AÚN ASÍ, LO QUE BUSCAMOS OBTENER SON
## LOS VALORES ESPERADOS DE LAS TASAS NOMINALES ANUALES CONFORMADAS POR EL ICP, ES DECIR, TNA, LAS CUALES SE MUESTRAN
## EN LA VARIABLE VALOR_TASA.

(Tasas_esperadas,tablaql) = set_expected_rate(Qcf.QCDate(4,10,2019), ois, zcc, fixing,Qcf.QCDate(4, 1, 2020))

Tasas_esperadas

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2019-10-04,2020-10-05,2020-10-05,"-100,000,000.00",0.00,True,"-70,555.63",CLP,0.018200,0.018213,0.0700%,"-71,361.11",0.0000%,1.000000,LinAct360
1,2020-10-05,2021-10-04,2021-10-04,"-100,000,000.00",0.00,True,"-43,217.06",CLP,0.018213,0.018221,0.0400%,"-40,444.44",0.0000%,1.000000,LinAct360
2,2021-10-04,2022-10-04,2022-10-04,"-100,000,000.00",0.00,True,"-111,324.97",CLP,0.018221,0.018241,0.1100%,"-111,527.78",0.0000%,1.000000,LinAct360
3,2022-10-04,2023-10-04,2023-10-04,"-100,000,000.00",0.00,True,"-277,864.25",CLP,0.018241,0.018292,0.2700%,"-273,750.00",0.0000%,1.000000,LinAct360
4,2023-10-04,2024-10-04,2024-10-04,"-100,000,000.00",0.00,True,"-469,204.12",CLP,0.018292,0.018378,0.4600%,"-467,666.67",0.0000%,1.000000,LinAct360
5,2024-10-04,2025-10-06,2025-10-06,"-100,000,000.00","-100,000,000.00",True,"-100,701,455.65",CLP,0.018378,0.018506,0.6900%,"-703,416.67",0.0000%,1.000000,LinAct360


In [11]:
#FUNCIÓN QUE ENTREGA LOS VALORES PRESENTE DE CADA PATA Y EL TOTAL.
def present_value(val_date,ois_cashflow,zcc) -> float:
    vpresente = Qcf.PresentValue()
    get_ois_sofr = partial(aux.get_ois_using_template, aux.type_ois_template, aux.TypeOis.SOFR)
    op = get_ois_sofr(
    rp=Qcf.RecPay.RECEIVE,
    notional=ois.notional,
    start_date=ois.start_date,
    tenor=Qcf.Tenor('6Y'),
    fixed_rate_value=.01,
    spread=ois.spread,
    gearing=ois.gearing)
    vp_fija = vpresente.pv(val_date, op[0], zcc)
    vp_flot = vpresente.pv(val_date, op[1], zcc)
    vp_final= vp_fija + vp_flot
    return (vp_fija), (vp_flot), (vp_final)


In [12]:
Valor_presente=present_value(Qcf.QCDate(4,10,2019),ois,zcc)
print(f"Valor presente pata fija (Recibir): {Valor_presente[0]}")
print(f"Valor presente pata flotante (Pagar): {Valor_presente[1]}")
print(f"Valor presente total : {Valor_presente[2]}")


Valor presente pata fija (Recibir): 104402782.93707068
Valor presente pata flotante (Pagar): -98344266.3352492
Valor presente total : 6058516.601821482


## Tests

In [13]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [14]:
amount_tol = 10000
rate_tol = .001

In [15]:
exitos = 0

In [16]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [19]:
ois = OisCashFlow(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [20]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [21]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [22]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [23]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [24]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued rate: {accrued_rate:.8%}')

ValueError: arrays must all be same length

Se verificará el cálculo usando `df_fixings`.

In [25]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [26]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [27]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [28]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [29]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [30]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [31]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [32]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [33]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [34]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [35]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [36]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

NameError: name 'accrued_rate' is not defined

In [37]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

NameError: name 'accrued_rate' is not defined

### `OisCashflow.accrued_interest`

In [38]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

AttributeError: 'OisCashFlow' object has no attribute 'tasa_eq'

In [39]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

NameError: name 'accrued_rate' is not defined

In [40]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

NameError: name 'accrued_interest' is not defined

### `OisCashflow.amount`

In [41]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

AttributeError: 'OisCashFlow' object has no attribute 'tasa_eq'

In [42]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [43]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

NameError: name 'amount' is not defined

In [44]:
suma_exito(amount, check_amount, 'monto')

NameError: name 'amount' is not defined

### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [45]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [46]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [47]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [48]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [52]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings, ois.end_date)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.06121277%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [53]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings, ois.end_date)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

IndexError: single positional indexer is out-of-bounds

In [54]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings, ois.end_date)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

IndexError: single positional indexer is out-of-bounds

In [55]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings, ois.end_date)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%


IndexError: single positional indexer is out-of-bounds

In [56]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings, ois.end_date)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%


IndexError: single positional indexer is out-of-bounds

### `present_value`

In [57]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings, ois.end_date)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: unsupported format string passed to tuple.__format__

In [58]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings, ois.end_date)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

IndexError: single positional indexer is out-of-bounds

In [59]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings, ois.end_date)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

IndexError: single positional indexer is out-of-bounds

In [60]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings, ois.end_date)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: unsupported format string passed to tuple.__format__

In [61]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings, ois.end_date)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: unsupported format string passed to tuple.__format__

In [62]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 1.5
